## Example sPlot  to create signal weights to be used in subsequent fit

The weights will be stored in a ROOT file as part of a HS::Weights object. They can then be combined with the data on the fly when performing subsequent fits. It is also possible to merge with the data tree :
    
       Weights wgts;
       wgts.LoadSaved("filename.root","HSsWeights");
       wgts.AddToTree(tree);
       
If this is attempted in interactive ROOT you will also need to have loaded the classes (next line)

First load the hs fit classes

In [ ]:
gROOT->ProcessLine(".x $HSCODE/hsfit/LoadFit.C+")

### Construct an sPlot object for running the fit

Set an output directory for saving weights and plots and stdout. It is best to provide the full path as this can then be used with PROOF as well.

I use the pwd variable to keep everything in the local directory, while applying the full path for PROOF.

In [ ]:
sPlot RF;
TString pwd = TString(gSystem->Getenv("PWD"))+"/";
RF.SetUp().SetOutDir(pwd+"sPlotFit");

### Set an observables

These are the fit variables. In general you can fit in as many dimensions as you need by supplying more variables. 

Here we are going to fit the missing mass (Mmiss) between 0-10


In [ ]:
///////////////////////////////Load Variables
RF.SetUp().LoadVariable("Mmiss[0,10]");//should be same name as variable in tree

IMPORTANT here we set the event ID variable. Each event in your tree should have a unique id with which to synchronise with weights.
    This is useful as it allows us to break the data up, perform seperate fits then combine the weights afterwards.
    The weights are not written into the input tree, rather they are stored seperately (as HS::Weights) and combined when required. This allows you to use various sets of weights when performing fits or regenerate the weights after some corrections etc.

In [ ]:
RF.SetUp().SetIDBranchName("fgID"); //this is set in Model1.C

### Create a signal PDF

The LoadSpecies Line adds the PDF Signal to the Extended Maximum Likelihood fit and creates a corresponding yield parameter Yld_Signal which will give the number of signal events in the data after the fit.
Note, I did not have to directly load this PDF I could have created more PDFs and combined them to make a more complicated PDF shape first.

We are just going to use a signal Gaussain with initial guess of mean 4 (with possible range 2-8), and sigma 1 (range 0.1-5)

In [ ]:
//////////////////////////////Make signal PDF
RF.SetUp().FactoryPDF("Gaussian::Signal( Mmiss, mean[4,2,8], sigma[1,0.1,5] )");
RF.SetUp().LoadSpeciesPDF("Signal",1);

### Create a background PDF

Here we just use a quadratic Chebychev polynomial to describe a smooth background. You can add extra a2,a3,etc. to increase the degree of the polynomial

In [ ]:
//////////////////////////////Make background PDF
RF.SetUp().FactoryPDF("Chebychev::BG(Mmiss,{a0[-0.1,-1,1],a1[0.1,-1,1]})");
RF.SetUp().LoadSpeciesPDF("BG",1);

### Define Bins/Splits
Often you wil want to split the data into distinct bins, in invariant mass or angle for example, and perform seperate fits to each bin. Here we can set as many different variables to split the data on as we like. Here we must use the Binner class accessed via RF.Bins() to propogate the splits into data and simulated events.

The function LoadBinVars takes a variable to make bins out of which has to be in the data tree, the number of bins to make and the limits of the variable. Alternatively you can supply the variable name, number of bins and an array of bin edges as for TAxis.

In this example we will not use splits, but you can test it by commenting out the line below. In this case after performing the individaul fits all weights will be combined into 1 file OUTDIR/Tweights.root

In [ ]:
//RF.Bins().LoadBinVar("Eg",5,3,4);

### Load Data


This just takes the treename "MyModel" and data file "Data.root" which were created in GenerateData

In [ ]:
///////////////////////////Load Data
RF.LoadData("MyModel","Data.root");

Turn on javascript root for interactive plots (even when saved as html file!)

In [ ]:
%jsroot

### Run the fit.
The test output will be redirected to your specified output directory (see code cell 2) as file logRooFit.txt. This will give full details of the fit.
Here we will just report the final result, the weights and the plots of fit PDF overlayed on the data as well as the fit residuals and pulls.

There are different options for running. 
To just run standard use Here::Go and the fits will just be performed within this interactive session, sequentially in the case of bins.

Here::One(&RF,i) will run one singal fit from the different bins produce with LoadBinVars, where i gives the index of the bin

Use Proof::Go(&RF,N) and the fits will be run in parallel via PROOF with each split/bin running on a different core, where N = number of cores to use.
    

In [ ]:
Here::Go(&RF);
//Here::One(&RF,3);
//Proof::Go(&RF,1);

### Draw some weighted variables

I can now use the resulting weights to make sPlots of the other variables in the tree, disentangling signal from background. 
I use the DrawWeighted function which takes standard TTree::Draw option for first argument and the weight species with the second (i.e. here I could plot Signal or BG) .

Additionally you can add cut condition with the third argument and a hist draw option with the fourth.

Here we can see the background subtracted \phi distributions for the 2 different polarisation states.

If splits were defined with LoadBinVars, then calling draw weighted automatically calls sPlot::MergeWeights() which combines all the weights into 1 file so they can be used on the full original tree. You may want to call MErgeWeights in the case you do not want to draw anything so at least the merged weights file is produced.



In [ ]:
auto* can=new TCanvas;
can->Divide(2,1);
can->cd(1);
TH1F* hPhi=new TH1F("hPhi","hPhi",100,-180,180);
RF.DrawWeighted("Phi>>hPhi","Signal");
RF.DrawWeighted("Phi","Signal","PolState==-1","same");
RF.DrawWeighted("Phi","Signal","PolState==1","same");
hPhi->SetMinimum(0);

can->cd(2);
RF.DrawWeighted("M1>>hM1(100,0,10)","Signal");
RF.DrawWeighted("M1","BG","","same");

can->Draw();

Now we can proceed to the main fit and use the sWeights  with the [Fit notebook](Fit.ipynb)